<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Global-Import" data-toc-modified-id="Global-Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Global Import</a></span></li><li><span><a href="#Jstson-Device" data-toc-modified-id="Jstson-Device-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Jstson Device</a></span><ul class="toc-item"><li><span><a href="#Target-System-Info" data-toc-modified-id="Target-System-Info-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://github.com/jetsonhacks/jetsonUtilities" rel="nofollow" target="_blank">Target System Info</a></a></span></li><li><span><a href="#Setup-Software-Environment" data-toc-modified-id="Setup-Software-Environment-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup Software Environment</a></span></li></ul></li><li><span><a href="#X86-Host-(TODO)" data-toc-modified-id="X86-Host-(TODO)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>X86 Host (TODO)</a></span><ul class="toc-item"><li><span><a href="#Host-System-Info" data-toc-modified-id="Host-System-Info-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Host System Info</a></span></li><li><span><a href="#Install-QEMU" data-toc-modified-id="Install-QEMU-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span><a href="https://github.com/NVIDIA/nvidia-docker/wiki/NVIDIA-Container-Runtime-on-Jetson#building-jetson-containers-on-an-x86-workstation-using-qemu" rel="nofollow" target="_blank">Install QEMU</a></a></span></li><li><span><a href="#Install-Nvidia-SDK-Manager-(Docker)" data-toc-modified-id="Install-Nvidia-SDK-Manager-(Docker)-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span><a href="https://developer.nvidia.com/nvidia-sdk-manager" rel="nofollow" target="_blank">Install Nvidia SDK Manager (Docker)</a></a></span></li><li><span><a href="#Cross-Compile-Tensort" data-toc-modified-id="Cross-Compile-Tensort-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span><a href="https://github.com/NVIDIA/TensorRT/tree/release/8.0#setting-up-the-build-environment" rel="nofollow" target="_blank">Cross-Compile Tensort</a></a></span></li><li><span><a href="#Jetson-Containers-🔗" data-toc-modified-id="Jetson-Containers-🔗-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Jetson Containers <a href="https://ngc.nvidia.com/catalog/Containers?orderBy=scoreDESC&amp;pageNumber=0&amp;query=jetson&amp;quickFilter=&amp;filters=" rel="nofollow" target="_blank">🔗</a></a></span></li></ul></li><li><span><a href="#Global-Configure" data-toc-modified-id="Global-Configure-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Global Configure</a></span></li><li><span><a href="#Onnx-to-Tensorrt" data-toc-modified-id="Onnx-to-Tensorrt-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Onnx to Tensorrt</a></span><ul class="toc-item"><li><span><a href="#Builder" data-toc-modified-id="Builder-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Builder</a></span></li><li><span><a href="#BuildConfig" data-toc-modified-id="BuildConfig-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>BuildConfig</a></span></li><li><span><a href="#Network-&amp;-OnnxParser" data-toc-modified-id="Network-&amp;-OnnxParser-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Network &amp; OnnxParser</a></span></li><li><span><a href="#Runtime" data-toc-modified-id="Runtime-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Runtime</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>References</a></span></li><li><span><a href="#FAQs" data-toc-modified-id="FAQs-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>FAQs</a></span></li></ul></div>

## Global Import

In [11]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 95))

import sys, os, io, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np

def _IMPORT(x):
    try:
        x = x.strip()
        if x.startswith('https://'):
            x = x[8:]
        if not x.endswith('.py'):
            x = x + '.py'
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        else:
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)

@register_line_cell_magic
def template_writefile(line, cell):
    path = os.path.dirname(line)
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
    with open(line, 'w') as fw:
        fw.write(cell.format(**globals()))

numpy 1.19.5
sklearn 0.0
pandas 1.1.5
ipywidgets 7.6.3
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
plotly 5.3.0
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
tflite not installed
onnx 1.10.1
onnxruntime 1.8.1
tensorrt not installed
tvm not installed


In [2]:
SCRIPT_ROOT = '/data/nb_data/jetson'

## Jstson Device

### [Target System Info][1]

[1]: https://github.com/jetsonhacks/jetsonUtilities

In [3]:
%%writefile $SCRIPT_ROOT/jetson_device_versions.sh

git clone --depth 1 https://github.com/jetsonhacks/jetsonUtilities.git
cd jetsonUtilities
python3 jetsonInfo.py

Overwriting /data/nb_data/jetson/jetson_device_versions.sh


```
NVIDIA Jetson Nano (Developer Kit Version)
 L4T 32.6.1 [ JetPack 4.6 ]
   Ubuntu 18.04.5 LTS
   Kernel Version: 4.9.253-tegra
 CUDA 10.2.300
   CUDA Architecture: 5.3
 OpenCV version: 4.1.1
   OpenCV Cuda: NO
 CUDNN: 8.2.1.32
 TensorRT: 8.0.1.6
 Vision Works: 1.6.0.501
 VPI: ii libnvvpi1 1.1.11 arm64 NVIDIA Vision Programming Interface library
 Vulcan: 1.2.70
```

In [12]:
# https://docs.nvidia.com/deeplearning/frameworks/install-tf-jetson-platform-release-notes/tf-jetson-rel.html#tf-jetson-rel
JP_VERSION = '46'
TF_VERSION = '2.5.0'
NV_VERSION = '21.08'

TRT_VERSION = 'release/8.0'

### Setup Software Environment

In [5]:
%%writefile $SCRIPT_ROOT/software_setup.sh

#!/bin/bash

USER_HOME=/home/$USER
CUDA_HOME=/usr/local/cuda

JP_VERSION=46
TF_VERSION=2.5.0
NV_VERSION=21.08

SUDO=''

PIP_INSTALL="$SUDO pip3 install --retries 10 --timeout 120"
APT_INSTALL="$SUDO apt install -y --no-install-recommends"
BOOTRC_FILE="$USER_HOME/.bootrc'
HOST_ADDR=`who | grep "pts/0" | awk -F" " '{match($NF, /[^(].*[^)]/); print substr($NF, RSTART,RLENGTH)}'`

echo_bootrc() {
    echo "$*" | tee -a $BOOTRC_FILE
}

echo_bashrc() {
    echo "$*" | tee -a $USER_HOME/.bashrc
}

# on boot script
__setup_bootrc() {
    echo -e "#!/bin/bash\nsleep 10" > $BOOTRC_FILE
    chmod 777 $BOOTRC_FILE
    sudo bash -c "echo '@reboot  root  test -x $BOOTRC_FILE && $BOOTRC_FILE' >> /etc/crontab"
}

# environment
__setup_bashrc() {
    echo_bashrc "alias pip='pip3'"
    echo_bashrc "alias python='python3'"
    echo_bashrc "export PIP_INSTALL='$PIP_INSTALL'"
    echo_bashrc "export APT_INSTALL='$APT_INSTALL'"
    echo_bashrc "export CUDA_HOME=$CUDA_HOME"
}

# mount.nfs host
__setup_mntnfs(){
    mkdir -p $USER_HOME/omega
    $APT_INSTALL nfs-common
    echo_bootrc "mount.nfs $HOST_ADDR:/blog/public $USER_HOME/omega"
}

# notebook
__setup_notebook() {
    $PIP_INSTALL autopep8
    $PIP_INSTALL jupyter jupyter_contrib_nbextensions jupyter_nbextensions_configurator
    $SUDO jupyter contrib nbextension install --sys-prefix
    $SUDO jupyter nbextensions_configurator enable
    git clone --depth 1 https://github.com.cnpmjs.org/qrsforever/jupyter_config.git
    $SUDO ./jupyter_config/install.sh
    echo_bootrc "su - $USER -c 'umask 0000; jupyter notebook --no-browser --notebook-dir=$USER_HOME/omega --allow-root --ip=0.0.0.0 --port=8118 &'"
}

# cmake
__setup_cmake() {
    wget https://github.com/Kitware/CMake/releases/download/v3.21.2/cmake-3.21.2-linux-aarch64.tar.gz
    tar zxf cmake-3.21.2-linux-aarch64.tar.gz
    cp -aprf cmake-3.21.2-linux-aarch64.tar/* /usr/
}

# machine learning
__setup_ml() {
    # tensorflow: https://docs.nvidia.com/deeplearning/frameworks/install-tf-jetson-platform/index.html#upgrading_tensorflow
    $APT_INSTALL libhdf5-serial-dev hdf5-tools libhdf5-dev zlib1g-dev zip libjpeg8-dev liblapack-dev libblas-dev gfortran
    $PIP_INSTALL -U pip testresources setuptools==49.6.0
    $PIP_INSTALL -U numpy==1.19.4 future==0.18.2 mock==3.0.5 keras_preprocessing==1.1.2 keras_applications==1.0.8 gast==0.4.0 futures protobuf pybind11 cython pkgconfig
    $SUDO env H5PY_SETUP_REQUIRES=0 && $PIP_INSTALL -U h5py==3.1.0
    $PIP_INSTALL -U --pre --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v${JP_VERSION} tensorflow==${TF_VERSION}+nv${NV_VERSION}
    
    # onnx2tensor
    $APT_INSTALL libprotobuf-dev protobuf-compiler
    git clone --depth 1 --recursive https://github.com.cnpmjs.org/onnx/onnx-tensorrt.git
    cd onnx-tensorrt && mkdir build && cd build && cmake ../ && make -j && sudo make install
    
    # pycuda, onnx_helper, graph
    $PIP_INSTALL --global-option=build_ext --global-option="-I$CUDA_HOME/include/" --global-option="-L$CUDA_HOME/lib" pycuda
    $PIP_INSTALL onnx_graphsurgeon --index-url https://pypi.ngc.nvidia.com
    $PIP_INSTALL polygraphy --index-url https://pypi.ngc.nvidia.com
    # git clone --depth 1 https://github.com.cnpmjs.org/NVIDIA/TensorRT.git
}

__setup_bootrc
__setup_bashrc
__setup_mntnfs

Overwriting /data/nb_data/jetson/software_setup.sh


## X86 Host (TODO)

### Host System Info

```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:00:06.0 Off |                    0 |
| N/A   28C    P0    46W / 250W |      0MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+
```

### [Install QEMU][1]

[1]: https://github.com/NVIDIA/nvidia-docker/wiki/NVIDIA-Container-Runtime-on-Jetson#building-jetson-containers-on-an-x86-workstation-using-qemu

In [6]:
%%writefile $SCRIPT_ROOT/qemu.sh

sudo apt-get install -y qemu binfmt-support qemu-user-static

# TODO FAQs: write error: Invalid argument
# copy multiarch/qemu-user-static:x86_64-aarch64:/usr/bin/qemu-aarch64-static ${pwd}
# run when reboot
docker run --rm --privileged multiarch/qemu-user-static --reset -p yes

# Test
# docker run --rm -v ${pwd}/qemu-aarch64-static:/usr/bin/qemu-aarch64-static -t arm64v8/ubuntu uname -m
docker run --rm -t arm64v8/ubuntu uname -m

Overwriting /data/nb_data/jetson/qemu.sh


### [Install Nvidia SDK Manager (Docker)](https://developer.nvidia.com/nvidia-sdk-manager)

In [7]:
%%writefile $SCRIPT_ROOT/sdkmanager_docker.sh

version=1.6.1
build=18175

docker load -i ./sdkmanager_$version.$build_docker.tar.gz
docker tag sdkmanager:$version.$build sdkmanager:latest
    
docker run -it --rm sdkmanager --help

docker run -itd --net host --name nvidia_sdkmanger --volume /data/:/data --entrypoint=/bin/bash sdkmanager 

# in container
# --host: Install Host Components
# sdkmanager --exitonfinish --cli downloadonly --logintype devzone --product Jetson --version 4.6 --targetos Linux \
#   --host --flash all --target JETSON_NANO_TARGETS--license accept --downloadfolder /data/docker/jetpack_files/

Overwriting /data/nb_data/jetson/sdkmanager_docker.sh


### [Cross-Compile Tensort](https://github.com/NVIDIA/TensorRT/tree/release/8.0#setting-up-the-build-environment)

In [19]:
%%template_writefile $SCRIPT_ROOT/cross_compile_tensort.sh

# Download and Update
git clone -b {TRT_VERSION} --depth 1 https://github.com/NVIDIA/TensorRT.git /data/TensorRT8.0
cd /data/TensorRT8.0
git submodule update --init --recursive

# Copy JetPack Files to docker
if [ ! -d docker/jetpack_files ]
then
    cp -aprf /data/docker/jetpack_files docker/
fi

./docker/build.sh --file docker/ubuntu-cross-aarch64.Dockerfile --tag tensorrt-jetpack-cuda10.2 --cuda 10.2

### Jetson Containers [🔗](https://ngc.nvidia.com/catalog/Containers?orderBy=scoreDESC&pageNumber=0&query=jetson&quickFilter=&filters=)

In [8]:
%%template_writefile $SCRIPT_ROOT/Dockerfile.l4t-tensorrt-tf2.5

FROM nvcr.io/nvidia/
    
ENV TZ=Asia/Shanghai \
    H5PY_SETUP_REQUIRES=0 \
    LANG=C.UTF-8 \
    PYTHONIOENCODING=utf-8 \
    DEBIAN_FRONTEND=noninteractive \
    APT_INSTALL="apt install -y --no-install-recommends" \
    PIP_INSTALL="python3 -m pip install --no-cache-dir --retries 20 --timeout 120" \
    PIP_INSTALL_FAST="python3 -m pip install --trusted-host mirrors.aliyun.com --extra-index-url http://mirrors.aliyun.com"

RUN rm -rf /usr/bin/python /usr/bin/python3

RUN apt update --fix-missing && rm -rf /usr/bin/python /usr/bin/python3 && \
    apt remove --purge python3.8 python3.8-dev && ln -s /usr/bin/python3.6 /usr/bin/python3 \
    $APT_INSTALL net-tools locate libhdf5-serial-dev hdf5-tools libhdf5-dev zlib1g-dev zip \
        libjpeg8-dev liblapack-dev libblas-dev gfortran python3.6 && \
    python3 -m pip uninstall tensorflow tensorflow-estimator tensorboard && \
    $PIP_INSTALL -U pip testresources setuptools==49.6.0 && \
    $PIP_INSTALL -U numpy==1.19.4 future==0.18.2 mock==3.0.5 grpcio==1.34.1 keras_preprocessing==1.1.2 \
        keras_applications==1.0.8 gast==0.4.0 futures protobuf pybind11 cython pkgconfig h5py==2.10.0 \
    $PIP_INSTALL -U --pre --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v{JP_VERSION} tensorflow=={TF_VERSION}+nv{NV_VERSION}

RUN cp /etc/apt/sources.list /etc/apt/sources.list.save && \
    sed -i s@http://ports.ubuntu.com@http://mirrors.aliyun.com@g /etc/apt/sources.list && \
    apt update && \
    $PIP_INSTALL_FAST onnx onnxruntime
        
# Error: $PIP_INSTALL -U h5py==3.1.0

## Global Configure

In [10]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE)
EXPLICIT_BATCH = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

onnx_file_path = '../onnx/repnet/saved_model.onnx'
engine_file_path = './saved_model.trt'

_DIR_(trt)

ModuleNotFoundError: No module named 'tensorrt'

In [ ]:
_DIR_(trt.BuilderFlag)

In [ ]:
for attr in _DIR_(trt.BuilderFlag, False):
    obj = eval('trt.BuilderFlag.%s' % attr)
    if isinstance(obj, trt.BuilderFlag):
        print('%50s:\t%d' % (attr, int(obj)))

## Onnx to Tensorrt

### Builder

In [ ]:
builder = trt.Builder(TRT_LOGGER)
_DIR_(builder)

In [ ]:
builder.max_batch_size, builder.platform_has_fast_fp16, builder.platform_has_fast_int8, builder.platform_has_tf32

### BuildConfig

In [ ]:
config = builder.create_builder_config()
_DIR_(config)

In [ ]:
config.max_workspace_size, config.flags, config.default_device_type, config.engine_capability

In [ ]:
config.max_workspace_size = 1 << 28
config.set_flag(trt.BuilderFlag.FP16)
config.set_flag(trt.BuilderFlag.STRICT_TYPES)

### Network & OnnxParser

In [ ]:
network = builder.create_network(EXPLICIT_BATCH)
_DIR_(network)

In [ ]:
parser = trt.OnnxParser(network, TRT_LOGGER)
_DIR_(parser)

In [ ]:
with open(onnx_file_path, 'rb') as model:
    if not parser.parse(model.read()):
        for e in range(parser.num_errors):
            print(parser.get_error(e))
        raise TypeError("Parser parse failed.")

In [ ]:
import onnx
import onnx_graphsurgeon as gs

In [ ]:
_DIR_(gs)

In [ ]:
graph = gs.import_onnx(onnx.load(onnx_file_path))

In [ ]:
node0 = graph.nodes[0]

In [ ]:
node0, node0.attrs

In [ ]:
node8 = graph.nodes[8]

In [ ]:
'to' in list(node8.attrs.keys())

In [ ]:
for node in graph.nodes:
    if 'allowzero' in list(node.attrs.keys()):
        print(node.attrs)

In [ ]:
_DIR_(graph)

In [ ]:
node2 = graph.nodes[2]

In [ ]:
_DIR_(node2)

In [ ]:
_DIR_(gs.Graph)

### Runtime

In [ ]:
runtime = trt.Runtime(TRT_LOGGER)
_DIR_(runtime)

## References

- [Running and Building ARM Docker Containers on x86][1]
- [NVIDIA L4T is a Linux based software distribution for the NVIDIA Jetson embedded computing platform][2]
- [Jetson Docker build][3]
- [This guide provides instructions for installing TensorFlow for Jetson Platform][4]
- [Jetson Docker Containers][5]
- [NVIDIA Container Runtime on Jetson][6]
- [Install Tensorrt][7]
- [Tensorrt Guide][11]
- [The following development platforms are supported with NVIDIA SDK Manager: Target Devices][8]
- [JetPack SDK in Docker for simple and clean flashing of a Jetson TX2][9]
- [Creating a Cross-Compilation like process for Jetson TX2 (aarch64) using QEMU with Docker][10]

[11]: https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html
[10]: https://trn84.medium.com/creating-a-cross-compilation-like-process-for-jetson-tx2-aarch64-using-qemu-with-docker-5cb38f6a65bc
[9]: https://trn84.medium.com/jetpack-sdk-in-docker-for-simple-and-clean-flashing-of-a-jetson-tx2-32b4db6c8d65
[8]: https://docs.nvidia.com/sdk-manager/system-requirements/index.html
[7]: https://docs.nvidia.com/deeplearning/tensorrt/quick-start-guide/index.html
[6]: https://github.com/NVIDIA/nvidia-docker/wiki/NVIDIA-Container-Runtime-on-Jetson#building-jetson-containers-on-an-x86-workstation-using-qemu
[5]: https://ngc.nvidia.com/catalog/Containers
[4]: https://docs.nvidia.com/deeplearning/frameworks/install-tf-jetson-platform/index.html
[3]: https://github.com/dusty-nv/jetson-containers
[2]: https://ngc.nvidia.com/catalog/containers/nvidia:l4t-base
[1]: https://www.stereolabs.com/docs/docker/building-arm-container-on-x86/


## FAQs

- [Attribute not found: allowzero][1]
- [Assertion failed: inputs.at(1).is_weights() && "The input pads is required to be an initializer."][2]
- [Stand-alone pad operation fails with: Assertion failed: inputs.at(1).is_weights()][3]
- [Assertion failed: convertOnnxPadding(onnxPadding, &begPadding, &endPadding) && "This version of TensorRT only supports padding on the outer two dimensions!"][4]
- [ONNX to TRT Error: Myelin Error in addNodeToMyelinGraph: 0 gather operation not supported within a loop body. ][5]
- [Build Tensorrt: cub/cub.cuh: No such file or directory #8][6]

<div class="alert alert-warning">
<code>
    if(NOT CUB_ROOT_DIR)
      if (CUDA_VERSION VERSION_LESS 11.0)
        set(CUB_ROOT_DIR ${CMAKE_CURRENT_SOURCE_DIR}/third_party/cub CACHE STRING "directory of CUB installation")
      endif()
    endif()
    /* fixit -DTRT_PLATFORM_ID=aarch64 -DCUDA_VERSION=10.2 */
</code>
</div>
   
[6]: https://githubmemory.com/repo/grimoire/amirstan_plugin/issues/8
[5]: https://github.com/NVIDIA/TensorRT/issues/1081
[4]: https://github.com/onnx/onnx-tensorrt/issues/533
[3]: https://github.com/onnx/onnx-tensorrt/issues/411
[2]: https://github.com/NVIDIA/TensorRT/issues/439
[1]: https://github.com/onnx/onnx/issues/3225